In [12]:
# Creates new business-level conceptual entities into the DB from an excel sheet
import sqlite3
import pandas as pd
import numpy as np
import os
from datetime import datetime
import sys
sys.path.append('../../')
from user_packages import hashing

In [13]:
# Set script variables
source_file_name = 'Org_Dictionary(Business Areas).csv'
source_file_path = '../../imports/business_area'

record_source = 'BusinessAreaCSV'

In [14]:
conn = sqlite3.connect('../../full_metadata.db')
cur = conn.cursor()

In [25]:
# read table to df
df = pd.read_csv(os.path.join(source_file_path, source_file_name))
df = df.replace({np.nan: None})

#df

In [23]:
# Create additional fields
df['RecordSource'] = record_source
df['LoadDate'] = datetime.now()

df['BusinessAreaKeyPhrase'] = df.apply(lambda row: row['BusinessName'] + '.' + row['BusinessAreaName'], axis=1)
df['ModelBusinessAreaKeyPhrase'] = df.apply(lambda row: row['BusinessAreaModelKeyPhrase'] + ':' + row['BusinessAreaKeyPhrase'], axis=1)
df['BusinessBusinessAreaKeyPhrase'] = df.apply(lambda row: row['BusinessName'] + ':' + row['BusinessAreaKeyPhrase'], axis=1)


In [24]:
# Hash fields

df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'BusinessHashKey'
  , columns = ['BusinessName']
)

df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'ModelHashKey'
  , columns = ['BusinessAreaModelKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'BusinessAreaHashKey'
  , columns = ['BusinessAreaKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'ModelBusinessAreaHashKey'
  , columns = ['ModelBusinessAreaKeyPhrase']
)

df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'BusinessBusinessAreaHashKey'
  , columns = ['BusinessBusinessAreaKeyPhrase']
)

df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'HashDiff'
  , columns = ['BusinessAreaName', 'DisplayName', 'Description', 'isDeleted', 'BusinessContact']
)

#df

In [14]:
# Clear down and Write to staging
conn.execute("DELETE FROM stg_Py_CSVToBusinessArea")
conn.commit()
df.to_sql('stg_Py_CSVToBusinessArea', conn, if_exists='append', index=False)

11

In [9]:
# Write to the Hub (BusinessArea)
sql_query = """
INSERT INTO rv_h_BusinessArea
(
    BusinessAreaHashKey
  , LoadDate
  , RecordSource
  , BusinessAreaKeyPhrase
)
SELECT DISTINCT
    BusinessAreaHashKey
  , LoadDate
  , RecordSource
  , BusinessAreaKeyPhrase
FROM
  stg_Py_CSVToBusinessArea
WHERE
  BusinessAreaHashKey NOT IN (SELECT BusinessAreaHashKey FROM rv_h_BusinessArea)
""";
conn.execute(sql_query)
conn.commit()

In [10]:
# Write to the Satellite (BusinessArea)
sql_query = """
INSERT INTO rv_s_BusinessArea
(
    BusinessAreaHashKey
  , LoadDate
  , RecordSource
  , HashDiff
  , BusinessAreaName
  , DisplayName
  , Description
  , isDeleted
  , BusinessContact
)
SELECT DISTINCT
    stg.BusinessAreaHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.HashDiff
  , stg.BusinessAreaName
  , stg.DisplayName
  , stg.Description
  , stg.isDeleted
  , stg.BusinessContact
FROM
  stg_Py_CSVToBusinessArea AS stg
  LEFT OUTER JOIN rv_s_BusinessArea AS sat ON (
    stg.BusinessAreaHashKey = sat.BusinessAreaHashKey
    AND sat.LoadDate = (
      SELECT MAX(z.LoadDate)
      FROM rv_s_BusinessArea AS z
      WHERE z.BusinessAreaHashKey = sat.BusinessAreaHashKey
    )
  )
WHERE
  (
    sat.HashDiff != stg.HashDiff
    OR sat.HashDiff IS NULL
  )
""";
conn.execute(sql_query)
conn.commit()

In [11]:
# Write to the Link (Model - BusinessArea)
sql_query = """
INSERT INTO rv_l_ModelBusinessArea
(
  ModelBusinessAreaHashKey
  , LoadDate
  , RecordSource
  , ModelHashKey
  , BusinessAreaHashKey
)
SELECT
  stg.ModelBusinessAreaHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.ModelHashKey
  , stg.BusinessAreaHashKey
FROM
  stg_Py_CSVToBusinessArea stg
WHERE
  stg.ModelBusinessAreaHashKey IS NOT NULL
  AND ModelBusinessAreaHashKey NOT IN (
    SELECT ModelBusinessAreaHashKey
    FROM rv_l_ModelBusinessArea
  )
""";
conn.execute(sql_query)
conn.commit()

In [15]:
# Write to the Link (Business - BusinessArea)
sql_query = """
INSERT INTO rv_l_BusinessBusinessArea
(
  BusinessBusinessAreaHashKey
  , LoadDate
  , RecordSource
  , BusinessHashKey
  , BusinessAreaHashKey
)
SELECT
  stg.BusinessBusinessAreaHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.BusinessHashKey
  , stg.BusinessAreaHashKey
FROM
  stg_Py_CSVToBusinessArea stg
WHERE
  stg.BusinessBusinessAreaHashKey IS NOT NULL
  AND BusinessBusinessAreaHashKey NOT IN (
    SELECT BusinessBusinessAreaHashKey
    FROM rv_l_BusinessBusinessArea
  )
""";
conn.execute(sql_query)
conn.commit()